In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf  # Add this line to import TensorFlow
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
# Load the preprocessed data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Set up image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='dataset/preprocessed/train',
    x_col='Name',
    y_col='HeadCount',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

Found 8186 validated image filenames.
Found 2047 validated image filenames.


In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Concatenate, Input, Activation
from tensorflow.keras.models import Model

def fire_module(x, squeeze_filters, expand_filters):
    squeeze = Conv2D(squeeze_filters, (1, 1), padding='same', activation='relu')(x)
    expand_1x1 = Conv2D(expand_filters, (1, 1), padding='same', activation='relu')(squeeze)
    expand_3x3 = Conv2D(expand_filters, (3, 3), padding='same', activation='relu')(squeeze)
    return Concatenate()([expand_1x1, expand_3x3])

def squeezenet_model(input_shape):
    inputs = Input(input_shape)

    # Initial convolution layer
    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    # Fire modules
    x = fire_module(x, squeeze_filters=16, expand_filters=64)
    x = fire_module(x, squeeze_filters=16, expand_filters=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    x = fire_module(x, squeeze_filters=32, expand_filters=128)
    x = fire_module(x, squeeze_filters=32, expand_filters=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    x = fire_module(x, squeeze_filters=48, expand_filters=192)
    x = fire_module(x, squeeze_filters=48, expand_filters=192)
    x = fire_module(x, squeeze_filters=64, expand_filters=256)
    x = fire_module(x, squeeze_filters=64, expand_filters=256)

    # Final convolution layer
    x = Conv2D(1, (1, 1), padding='same', activation='linear')(x)
    x = GlobalAveragePooling2D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

model = squeezenet_model(input_shape=(224, 224, 3))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Save the trained model
model.save('head_count_squeezenet_model.h5')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 55, 55, 64)   0           ['conv2d[0][0]']                 
                                                                                              

/var/folders/qv/7r3vg1ns6qbgvs0t3_dxc3qm0000gn/T/ipykernel_2471/1437364992.py:43: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/50


2024-03-29 16:26:15.575790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-03-29 16:26:15.580419: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


256/256 [==============================] - ETA: 0s - loss: 18.3647 - mae: 2.9451

2024-03-29 16:27:26.639958: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


256/256 [==============================] - 76s 295ms/step - loss: 18.3647 - mae: 2.9451 - val_loss: 13.0787 - val_mae: 2.6228
Epoch 2/50
256/256 [==============================] - 76s 295ms/step - loss: 14.6636 - mae: 2.6410 - val_loss: 11.2965 - val_mae: 2.5449
Epoch 3/50
256/256 [==============================] - 75s 294ms/step - loss: 14.0600 - mae: 2.5691 - val_loss: 11.8161 - val_mae: 2.5698
Epoch 4/50
256/256 [==============================] - 76s 296ms/step - loss: 13.3293 - mae: 2.4908 - val_loss: 11.0152 - val_mae: 2.4424
Epoch 5/50
256/256 [==============================] - 76s 294ms/step - loss: 12.5807 - mae: 2.4142 - val_loss: 11.2871 - val_mae: 2.4656
Epoch 6/50
256/256 [==============================] - 76s 295ms/step - loss: 12.3919 - mae: 2.3975 - val_loss: 12.3164 - val_mae: 2.5353
Epoch 7/50
256/256 [==============================] - 76s 295ms/step - loss: 12.5009 - mae: 2.4127 - val_loss: 9.9103 - val_mae: 2.3013
Epoch 8/50
256/256 [==============================] -

In [6]:
from tensorflow.keras.models import load_model

loaded_model = load_model('head_count_squeezenet_model.h5')

In [7]:
train_loss, train_rmse = model.evaluate(train_generator)

print(f"Training Loss: {train_loss}")
print(f"Training RMSE: {train_rmse}")

2024-03-29 17:30:01.719506: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


256/256 [==============================] - 61s 237ms/step - loss: 9.4061 - mae: 2.1486
Training Loss: 9.40605354309082
Training RMSE: 2.148550510406494


In [8]:
test_df = pd.read_csv('test.csv')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='dataset/preprocessed/test',
    x_col='Name',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False 
)

Found 3963 validated image filenames.


In [9]:
predictions = model.predict(test_generator)

2024-03-29 17:31:13.707663: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


124/124 [==============================] - 10s 82ms/step


In [10]:
import numpy as np

# Round the predictions to the nearest integer
predicted_head_counts = np.round(predictions.flatten()).astype(int)

# Create a DataFrame for the submission
submission_df = pd.DataFrame({
    'Name': test_df['Name'],
    'HeadCount': predicted_head_counts
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_main3.csv', index=False)